# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import json
import re

# Load data

In [ ]:
# create a lookup for the pdf parse based on paper ID
corpus = {}
with open(f'/content/gdrive/MyDrive/Independent study - Max & Carlos/scifact_dataset/corpus.jsonl') as f_pdf:
    for line in f_pdf:
        pdf_parse_dict = json.loads(line)
        corpus[pdf_parse_dict['doc_id']] = pdf_parse_dict
print("Corpus parsed.")

Corpus parsed.


In [ ]:
claim_id_to_claim = []
with open(f'/content/gdrive/MyDrive/Independent study - Max & Carlos/scifact_dataset/claims_train.jsonl') as f_pdf:
    for line in f_pdf:
        claim_dict = json.loads(line)
        nei_ids = []
        for id in claim_dict['cited_doc_ids']:
            if str(id) not in claim_dict['evidence']:
                nei_ids.append(id)
        if not nei_ids:
          continue
        claim_dict['cited_doc_ids'] = nei_ids
        claim_dict['evidence'] = {}
        claim_id_to_claim.append(claim_dict)
with open(f'/content/gdrive/MyDrive/Independent study - Max & Carlos/scifact_dataset/claims_dev.jsonl') as f_pdf:
    for line in f_pdf:
        claim_dict = json.loads(line)
        nei_ids = []
        for id in claim_dict['cited_doc_ids']:
            if str(id) not in claim_dict['evidence']:
                nei_ids.append(id)
        if not nei_ids:
          continue
        claim_dict['cited_doc_ids'] = nei_ids
        claim_dict['evidence'] = {}
        claim_id_to_claim.append(claim_dict)
print("Claims parsed.")
print(len(claim_id_to_claim))

Claims parsed.
454


In [ ]:
# create a lookup for the pdf parse based on paper ID
citances = []
with open(f'/content/gdrive/MyDrive/Independent study - Max & Carlos/scifact_dataset/claims_with_citances.jsonl') as f_pdf:
    for line in f_pdf:
        citance_dict = json.loads(line)
        citances.append((re.sub(r' \[\d+\]', '', citance_dict['citance']), citance_dict['claims']))
print("Citances parsed.")

Citances parsed.


In [ ]:
claims_and_evidence = {}
for i in claim_id_to_claim:
    i['normalized_claim'] = re.sub(r'[.!?,\s]', '', i['claim'].lower())
    claims_and_evidence[i['normalized_claim']] = i['cited_doc_ids']

In [ ]:
citance_to_id_dict = {}
with open(f'/content/gdrive/MyDrive/Independent study - Max & Carlos/scifact_dataset/claims_with_citances.jsonl') as f_pdf:
    for line in f_pdf:
        citance_dict = json.loads(line)
        citance_to_id_dict[re.sub(r' \[\d+\]', '', citance_dict['citance'])] = citance_dict['s2orc_id']

#

In [ ]:
# Setup of NEI
outer = {}
orpus = {}
citances_to_id = {}
results = []
failed_keys = []
fk2boogaloo = 0
# Loop through citances dict
num = 0
for citance in citances:
    # For each citance, loop through all claims
    for claim in citance[1]:

        claim['normalized'] = re.sub(r'[.!?,\s]', '', claim['text'].lower())
        # Skip if there is no matching claim
        # if claim['normalized'] not in claims_and_evidence:
        #     failed_keys.append(claim['normalized'])
        #     continue
        foo = {}
        if claim['normalized'] in claims_and_evidence:
            ids = []
            for doc_id in claims_and_evidence[claim['normalized']]:
                ids.append(doc_id)
                orpus[int(doc_id)] = {
                            'title': corpus[int(doc_id)]['abstract'][0],
                            'abstract': corpus[int(doc_id)]['abstract'],
                            'doc_id': int(doc_id)
                        }
            foo['claim'] = citance[0]
            foo['evidence'] = {}
            foo['citance_id'] = citance_to_id_dict[citance[0]]
            foo['doc_ids'] = ids
            outer[citance[0]] = foo

print(fk2boogaloo)
print(len(failed_keys))

0
0


In [ ]:
results = list(outer.values())
obj = json.dumps(results, indent=4)
with open(f'/content/gdrive/MyDrive/Independent study - Max & Carlos/data/dataset/neis.json', 'w') as f:
  f.write(obj)

with open('/content/gdrive/MyDrive/Independent study - Max & Carlos/data/dataset/nei_corpus.jsonl', "w") as f:
    for item in list(orpus.values()):
        json_item = json.dumps(item)
        f.write(json_item + "\n")